In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SmartSkinCNN(nn.Module):
    def __init__(self, num_classes=19):
        super(SmartSkinCNN, self).__init__()

        self.conv_block1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.conv_block2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=2, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.conv_block3 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=2, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.conv_block4 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=2, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.flatten = nn.Flatten()
        conv_output_size = self._get_conv_output_shape()
        self.fc1 = nn.Linear(conv_output_size, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, num_classes)

    def _get_conv_output_shape(self):
        with torch.no_grad():
            x = torch.zeros(1, 3, 180, 180)  # Use your input image size here
            x = self.conv_block1(x)
            x = self.conv_block2(x)
            x = self.conv_block3(x)
            x = self.conv_block4(x)
            return x.view(1, -1).shape[1]

    def forward(self, x):
        x = self.conv_block1(x)
        x = self.conv_block2(x)
        x = self.conv_block3(x)
        x = self.conv_block4(x)
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

In [2]:
from torchsummary import summary
model = SmartSkinCNN(num_classes=19)
summary(model, input_size=(3, 180, 180))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 180, 180]           1,792
              ReLU-2         [-1, 64, 180, 180]               0
            Conv2d-3         [-1, 64, 180, 180]          36,928
              ReLU-4         [-1, 64, 180, 180]               0
         MaxPool2d-5           [-1, 64, 90, 90]               0
            Conv2d-6          [-1, 128, 91, 91]          32,896
              ReLU-7          [-1, 128, 91, 91]               0
            Conv2d-8          [-1, 128, 92, 92]          65,664
              ReLU-9          [-1, 128, 92, 92]               0
        MaxPool2d-10          [-1, 128, 46, 46]               0
           Conv2d-11          [-1, 256, 47, 47]         131,328
             ReLU-12          [-1, 256, 47, 47]               0
           Conv2d-13          [-1, 256, 48, 48]         262,400
             ReLU-14          [-1, 256,

In [3]:
from torchviz import make_dot
x = torch.randn(1, 3, 180, 180)
y = model(x)
make_dot(y, params=dict(model.named_parameters())).render("smartskin_cnn", format="jpg")

'smartskin_cnn.jpg'